In [ ]:
from openai import OpenAI
from google.adk.tools.base_tool import BaseTool
from google.adk.agents.llm_agent import Agent
import os
from dotenv import load_dotenv
from dataclasses import dataclass
from typing import List, Dict
from openai import InternalServerError

BASE_URL = "https://api.deepgen.app/v1"
MODEL = "gemini-2.0-flash"
load_dotenv()


class DeepGenTool(BaseTool):

    def __init__(self, model = MODEL):
        API_KEY = os.getenv("DEEPGEN_API_KEY")
        if not API_KEY:
            raise ValueError("DEEPGEN_API_KEY environment variable not set.")
        
        self.client = OpenAI(api_key=API_KEY, base_url=BASE_URL)
        self.model = model
    
    def chat(self, messages, model=None):
        model = model or self.model
        try:
            response = self.client.chat.completions.create(
                model=model,
                messages=messages
            )
            return response.choices[0].message.content
        except InternalServerError as e:
            # Essayer de voir le corps de la réponse du serveur
            print("STATUS CODE:", e.status_code if hasattr(e, "status_code") else "no status")
            if hasattr(e, "response") and e.response is not None:
                try:
                    print("RAW BODY:", e.response.text)
                except Exception:
                    print("RAW BODY (repr):", repr(e.response))
            raise
        except Exception as e:
            print("OTHER ERROR:", repr(e))
            raise

root_agent = Agent(
    tools = [DeepGenTool()],
    model = MODEL,
    name = "DeepGen_Agent"
)


STATUS CODE: 500
RAW BODY: Internal Server Error
ERREUR LORS DE L’APPEL À L’API :
InternalServerError('Internal Server Error')


In [1]:
import os
import requests
from dotenv import load_dotenv

load_dotenv()

API_KEY = os.getenv("DEEPGEN_API_KEY")
URL = "https://api.deepgen.app/o3-mini/api"

if __name__ == "__main__":
    if not API_KEY:
        raise ValueError("DEEPGEN_API_KEY manquante dans le .env")

    payload = {
        "question": "Where is Annecy",
        "max_tokens": 4096
    }

    headers = {
        "Authorization": f"Bearer {API_KEY}",
        "Content-Type": "application/json"
    }

    resp = requests.post(URL, json=payload, headers=headers)
    print("Status code:", resp.status_code)
    print("Raw body:", resp.text)

    # Si c'est du JSON:
    try:
        data = resp.json()
        print("JSON:", data)
    except Exception:
        pass


Status code: 200
Raw body: [{"error":"Sorry,  paid AI Credits are needed for this request.\nYou can purchase a new subscription or pack from [this page](https://deepgen.app/pricing)","language":"en","conversation_id":"99f21b89-0290-42ab-a5e4-b2772cd23ea0","group_id":"b233f724-f6a9-5a1c-8e94-5e7326f26689","output_message_id":"48fbfdee-acb2-4958-a87a-92c20ad3914e","input_message_id":"135ed63b-2280-4519-8b50-bb46bb3eba08","agent_alias":"o3-mini"}]
JSON: [{'error': 'Sorry,  paid AI Credits are needed for this request.\nYou can purchase a new subscription or pack from [this page](https://deepgen.app/pricing)', 'language': 'en', 'conversation_id': '99f21b89-0290-42ab-a5e4-b2772cd23ea0', 'group_id': 'b233f724-f6a9-5a1c-8e94-5e7326f26689', 'output_message_id': '48fbfdee-acb2-4958-a87a-92c20ad3914e', 'input_message_id': '135ed63b-2280-4519-8b50-bb46bb3eba08', 'agent_alias': 'o3-mini'}]
